In [1]:
import ipaddress
import re
import nltk
import requests
from bs4 import BeautifulSoup
import whois
from datetime import datetime
from six.moves import urllib
from urllib.parse import urlparse
from urllib.parse import urlencode
import pandas as pd
import tldextract
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

ModuleNotFoundError: No module named 'nltk'

In [2]:
data = pd.read_csv(r"urls_dataset.csv")
data.shape

(19800, 2)

In [3]:
data.head()

,url,type
0,http://ringring.vn/hai-thieu-nu-quan-khan-tang...,0
1,http://distrimarsanitarios.soydg.com/wp-conten...,1
2,http://pastehtml.com/view/be8tc2420.html,1
3,http://olx.ua/uk/list/q-%D0%B2%D0%B5%D0%BB%D0%...,0
4,http://www.stevenmissirlian.com/wp-content/the...,1


In [4]:
data["type"].value_counts()

0    9900
1    9900
Name: type, dtype: int64

In [5]:
data.isnull().values.any()

False

In [6]:
urls = data.iloc[:,0]
urls

0        http://ringring.vn/hai-thieu-nu-quan-khan-tang...
1        http://distrimarsanitarios.soydg.com/wp-conten...
2                 http://pastehtml.com/view/be8tc2420.html
3        http://olx.ua/uk/list/q-%D0%B2%D0%B5%D0%BB%D0%...
4        http://www.stevenmissirlian.com/wp-content/the...
                               ...                        
19795    http://bill-verizon.7billpay-aol.com/ForYourHo...
19796           http://www.anindyavoice.com/includes/dpbx/
19797                http://adverteaze.com/remax/index.htm
19798    http://www.pls-print.com/images/index.htm?ref=...
19799    http://support.account-updated.com/paypal/1069...
Name: url, Length: 19800, dtype: object

In [7]:
df = data

In [8]:
def getDomain(url):
    split_url = url.split("://");
    i = (0,1)[len(split_url)>1];
    dm = split_url[i].split("?")[0].split('/')[0].split(':')[0].lower();
    return dm

In [9]:
def havingIP(url):
    tokens=nltk.word_tokenize(url)
    tok=str(tokens).split('/')
    for i in range(len(tok)):
        #print(tok[i])
        if re.match("^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$",tok[i]):
            if ipaddress.ip_address(tok[i]):
                return 1
    return 0

In [10]:
def getLength(url):
    if len(url) < 54:
        length = 0            
    else:
        length = 1            
    return length

In [11]:
def getDepth(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth

In [12]:
def redirection(url):
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0


In [13]:
def httpsDomain(url):
    domain = urlparse(url).netloc
    if 'https' in domain:
        return 0
    else:
        return 1

In [14]:
pish_words= r"download|login|logout|signup|signin|signout|site|wp|view|js|images|redirect"\
            r"confirm|account|banking|webscr|secure|ebayisapi|admin|includes|content|track"\

# pish keywords in the url (0 legitimate, 1 malicious)
def havePishWord(url):
    match=re.search(pish_words,url)
    if match:
        return 1
    else:
        return 0

In [15]:
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"


# Shortening Services in URL (0 legitimate, 1 malicious)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [16]:
# length of longest word in the URL
def longest_word(url):
    l=[]
    tokens=nltk.word_tokenize(url)
    word_list=str(tokens).split('/')
    longest_word = ''  
    for word in word_list:
        l.append(len(word))
    return max(l)

In [17]:
# # Web traffic (0 legitimate, 1 malicious)
# def web_traffic(url):
#     try:
#         #Filling the whitespaces in the URL if any
#         url = urllib.parse.quote(url)
#         rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find("REACH")['RANK']
#         rank = int(rank)
#     except TypeError:
#         return 1
#     if rank <100000:
#         return 1
#     else:
#         return 0


In [18]:
# difference between termination time and creation time of domain (0 legitimate, 1 malicious)
def domainAge(domain_name):
    
    try: 
        creation_date =  whois.whois(domain_name).creation_date
        expiration_date = whois.whois(domain_name).expiration_date
   
    
        if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
            try:
                creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
                expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
            except:
                return 1
        if ((expiration_date is None) or (creation_date is None)):
            return 1
        elif ((type(expiration_date) is list) or (type(creation_date) is list)):
            return 1
        else:
            ageofdomain = abs((expiration_date - creation_date).days)
            if ((ageofdomain/30) < 6):
                age = 1
            else:
                age = 0
        return age  
    except:
        print("Domain doesnt exist in whois database ", domain_name)

In [ ]:
df["MU_dots"] = " "
df["MU_hypens"] = " "
df["MU_at"] = " "
df["MU_and"] = " "
df["MU_percentage"] = " "
df["MU_slash"] = " "
df["MU_star"] = " "
df["MU_comma"] = " "
df["MU_semicolon"] = " "
df["MU_dollar"] = " "
df["MU_colon"] = " "
df["MU_question"] = " "
df["MU_space"] = " "
df["MU_underscore"] = "_"
df["MU_equal"] = " "
df["MU_hash"] = " "
df["length"] = " "
df["having_ip"] = " "
df["depth"] = " "
df["redirection"] = " "
df["https_domain"] = " "
df["pish_word"] = " "
df["has_tinyURL"] = " "
df["length_longest_word"] = " "
# df["web_traffic"] = " "
df["age"] = " "





index = 0
for i in urls: 
    dots = i.count('.')
    hypens = i.count('-')
    at = i.count('@')
    And = i.count('&')
    percentage = i.count('%')
    slash = i.count('/')
    star = i.count('*')
    comma = i.count(',')
    semicolon = i.count(';')
    dollar = i.count('$')
    colon = i.count(":")
    question = i.count('?')
    space = i.count(' ')
    underscore = i.count('_')
    equal = i.count('=')
    hasH = i.count("#")
    
    
    
    dm = getDomain(i)
    ip = havingIP(i)
    length = getLength(i)
    depth = getDepth(i)
    redirect = redirection(i)
    httpsdomain = httpsDomain(i)
    has_phish_word = havePishWord(i)
    tiny_url = tinyURL(i)
    length_longestWord = longest_word(i)
#     webtraffic = web_traffic(i)
    domain_age = domainAge(dm)
#     i_frame = iframe(i)
    
    
    
#     split_url = urls[index].split("://");
#     i = (0,1)[len(split_url)>1];
#     dm = split_url[i].split("?")[0].split('/')[0].split(':')[0].lower();

    

#     if(is_registered(dm)): 
#       creation_date =  whois.whois(dm).creation_date
#       expired_date = whois.whois(dm).expiration_date
#       # print("Age of Domain: ", dm, " ", creation_date)

#       try: 
#         age = days(creation_date)
#         daysleft = days_left(expired_date)
#       except:
#         print("Url passed")
      

    df["MU_dots"][index] = dots
    df["MU_hypens"][index] = hypens
    df["MU_at"][index] = at
    df["MU_and"][index] = And
    df["MU_percentage"][index] = percentage
    df["MU_slash"][index] = slash
    df["MU_star"][index] = star
    df["MU_comma"][index] = comma
    df["MU_semicolon"][index] = semicolon
    df["MU_dollar"][index] = dollar
    df["MU_colon"][index] = colon
    df["MU_question"][index] = question
    df["MU_space"][index] = space
    df["MU_underscore"][index] = underscore
    df["MU_equal"][index] = equal
    df["MU_hash"][index] = hasH
    df["having_ip"][index] = ip
    df["length"][index] = length
    df["depth"][index] = depth
    df["redirection"][index] = redirect
    df["https_domain"][index] = httpsdomain
    df["pish_word"][index] = has_phish_word
    df["has_tinyURL"][index] = tiny_url
    df["length_longest_word"][index] = length_longestWord
#     df["web_traffic"][index] = webtraffic
    df["age"][index] = domain_age
    
    
    
   


    
    index+=1


# print(df)

Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  www.stevenmissirlian.com
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  restomaison.net
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  us.battle.net.ok.qq-rs.com
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify2.id-app7.net
Domain doesnt exist in whois database  extremreklam.com
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  configurazione-supporto-apple.store-contatta-ufficiale.maritat.com
Domain doesnt exist in whois database  fb.com.accounts.login.userid.336480.fbsbk.com
Domain

Domain doesnt exist in whois database  www.urukdesign.com
Domain doesnt exist in whois database  service.confirm.paypal.cmd.cgi-bin.2866sd4f2e654sfd4e6d52sd4sd3.cursodecorretordeimoveisembh.com
Domain doesnt exist in whois database  www.uniformespromotex.com
Domain doesnt exist in whois database  www.circuitomarcial.com
Domain doesnt exist in whois database  www.pahadika-tc.com
Domain doesnt exist in whois database  linuxeman.com
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify9.eu-id3.net
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify3.id-app7.net
Domain doesnt exist in whois database  1-configurazione-supporto-apple.store-contatta.bimabn.com
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  anglo-american-supplier.yolasite.com
Domain doesnt exist in whois database  www.zabouna.com
Error trying to

Domain doesnt exist in whois database  onlinebanking.bankofamerica.com.lordleandro.com
Domain doesnt exist in whois database  txellwebs.com
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify0.id8-eu.net
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify3.id-app7.co.uk
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  tinnhanh360.net
Domain doesnt exist in whois d

Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  update-information-paypals-accounts-supports.com.lubinhote.com
Domain doesnt exist in whois database  www.bimabn.com
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  1-configurazione-supporto-apple.store-contatta.bimabn.com
Domain doesnt exist in whois database  www.terapias-spa.com
Domain doesnt exist in whois database  www.pt-lidi.com
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  propertmartltd.com
Domain doesnt exist in whois database  www.happinesslyrics.com
Domain doesnt exist in whois database  luct.myqnapcloud.com
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  https.paypal

Domain doesnt exist in whois database  www.egyptshuttletravel.com
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify2.srv-woa1.co.uk
Domain doesnt exist in whois database  txellwebs.com
Domain doesnt exist in whois database  coloroid.ru
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  www.mobileclickbradesco.com
Domain doesnt exist in whois database  patricamz.com
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify3.srv-woa1.com
Error trying to connect to socket: closing socket
Error trying to connect to socke

...

Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket


Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket


Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  www.jrgonza

Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  us.battle.net-wow-net.d3-account.net
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  cloud4call.com
Domain doesnt exist in whois database  www.zgmmzzw.com
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  www.id8-eu.co.uk
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  us.battle.net-bizzard-d3.account-com.net
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  txellwebs.com
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to

Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify1.eu-id3.net
Domain doesnt exist in whois database  www.gahomeappliances.com
Domain doesnt exist in whois database  themusicbonanza.com
Domain doesnt exist in whois database  www.poledancerfishingcharters.com
Domain doesnt exist in whois database  appleid.apple.co.uk.cgi-bin.webobjects.myappleid.woa.verify5.id8-eu.net
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Domain doesnt exist in whois database  tinnhanh360.net
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to socket: closing socket
Error trying to connect to 

In [ ]:
df.to_csv("dataset_urls.csv")